In [2]:
# import libraries
#!pip install xgboost
import nltk
import pandas as pd
from sqlalchemy import create_engine
from nltk import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import make_pipeline, FeatureUnion,Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
#from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

#, CountVectorizer, TfidfTransformer
#from sklearn.feature_extraction.text import
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download(['stopwords','punkt','wordnet'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jobao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jobao\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jobao\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [40]:
d=df.drop(['id','message','original','genre'],axis=1)
d.melt(value_vars=d.columns)
d.astype(int).sum().sort_values()

child_alone                   0
offer                       118
shops                       120
tools                       159
fire                        282
hospitals                   283
missing_people              298
aid_centers                 309
clothing                    405
security                    471
cold                        530
electricity                 532
money                       604
search_and_rescue           724
military                    860
refugees                    875
other_infrastructure       1151
death                      1194
transport                  1201
medical_products           1313
buildings                  1333
other_weather              1376
water                      1672
infrastructure_related     1705
medical_help               2084
floods                     2155
shelter                    2314
storm                      2443
earthquake                 2455
food                       2923
other_aid                  3446
request 

In [70]:
d=pd.DataFrame(df.drop(['id','message','original','genre'], axis=1).astype(int).sum().sort_values(ascending=False).head(5)).reset_index()
d.columns=['Category','count']
d
#pd.DataFrame(data = d).sort_values(by=['Count'], ascending=False)['Count']

,Category,count
0,related,20094
1,aid_related,10860
2,weather_related,7297
3,direct_report,5075
4,request,4474


In [3]:
# load data from database
engine = create_engine('sqlite:///disaster_response')
df = pd.read_sql('SELECT * FROM database',con=engine)
df.loc[df['related'] == '2','related'] = '1'
X = df['message']
Y = df.iloc[:,4:]


In [7]:
def tokenize(text):
    tokens = word_tokenize(text)
    
    tok=[WordNetLemmatizer().lemmatize(tok, pos='v') for tok in tokens]
    tok=[tok for tok in tokens if tok not in stopwords.words("english")]
         
        
    return tokens

In [8]:
#!pip install xgboost
#from sklearn.feature_extraction.text import TfidfVectorizer
#from xgboost import XGBClassifier

# pipeline = make_pipeline(
#     TfidfVectorizer(),
#     MultiOutputClassifier(XGBClassifier())
# )
pipeline = Pipeline([ 
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
   ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

x_train,x_test,y_train,y_test = train_test_split(X,Y)
pipeline.fit(x_train,y_train)


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x0000017E296F7280>)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [8]:
y_pred=pipeline.predict(x_test)
from sklearn.metrics import classification_report,accuracy_score
for col in range(len(y_test.columns)):
    print('Category: {} '.format(y_test.columns[col]))
    print(classification_report(y_test.iloc[:, col].values, y_pred[:, col]))
    print('Accuracy {}\n\n'.format(accuracy_score(y_test.iloc[:, col].values, y_pred[:, col])))


Category: related 
              precision    recall  f1-score   support

           0       0.74      0.25      0.37      1543
           1       0.81      0.97      0.88      5011

    accuracy                           0.80      6554
   macro avg       0.77      0.61      0.63      6554
weighted avg       0.79      0.80      0.76      6554

Accuracy 0.801953005797986


Category: request 
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5413
           1       0.90      0.42      0.58      1141

    accuracy                           0.89      6554
   macro avg       0.90      0.71      0.76      6554
weighted avg       0.89      0.89      0.88      6554

Accuracy 0.8918217882209338


Category: offer 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6530
           1       0.00      0.00      0.00        24

    accuracy                           1.00      6554
   macro avg  

C:\Users\jobao\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jobao\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jobao\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.76      0.88      0.81      3838
           1       0.78      0.60      0.68      2716

    accuracy                           0.76      6554
   macro avg       0.77      0.74      0.74      6554
weighted avg       0.76      0.76      0.76      6554

Accuracy 0.7625877326823314


Category: medical_help 
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      6031
           1       0.64      0.06      0.10       523

    accuracy                           0.92      6554
   macro avg       0.78      0.53      0.53      6554
weighted avg       0.90      0.92      0.89      6554

Accuracy 0.9221849252364968


Category: medical_products 
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6239
           1       0.72      0.06      0.11       315

    accuracy                           0.95      6554
   macro avg    

C:\Users\jobao\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jobao\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jobao\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jobao\Anaconda3\lib\site-packages\sklearn\metr

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6525
           1       0.00      0.00      0.00        29

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554

Accuracy 0.995575221238938


Category: hospitals 
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6485
           1       0.00      0.00      0.00        69

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

Accuracy 0.989472078120232


Category: shops 


C:\Users\jobao\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jobao\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jobao\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jobao\Anaconda3\lib\site-packages\sklearn\metr

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       0.00      0.00      0.00        31

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554

Accuracy 0.9952700640830028


Category: aid_centers 
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6478
           1       0.00      0.00      0.00        76

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

Accuracy 0.9882514494964907


Category: other_infrastructure 
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6298
           1       0.00      0.00      0.00       256

    accuracy                           0.96      6554
   macro avg 

C:\Users\jobao\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jobao\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jobao\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6483
           1       0.00      0.00      0.00        71

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

Accuracy 0.9891669209642966


Category: earthquake 
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      5974
           1       0.91      0.70      0.79       580

    accuracy                           0.97      6554
   macro avg       0.94      0.85      0.89      6554
weighted avg       0.97      0.97      0.97      6554

Accuracy 0.9671956057369545


Category: cold 
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6418
           1       0.82      0.07      0.12       136

    accuracy                           0.98      6554
   macro avg       0.90      0

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.utils import parallel_backend

parameters =  {'clf__estimator__min_samples_leaf': [1,2],'clf__estimator__n_estimators': [10,15]}

cv = GridSearchCV(pipeline, parameters,verbose=3, n_jobs=-1)
with parallel_backend('multiprocessing'): 
    cv.fit(x_train, y_train)

    print(cv.best_params_)
{'clf__estimator__n_estimators': 15}

Fitting 5 folds for each of 4 candidates, totalling 20 fits


In [18]:
from sklearn.model_selection import RandomizedSearchCV
from time import time
parameters =  {'clf__estimator__min_samples_leaf': [1,2],'clf__estimator__n_estimators': [10,15,20]}

n_iter_search = 15
random_search = RandomizedSearchCV(
    pipeline, param_distributions=parameters, n_iter=n_iter_search
)

start = time()
random_search.fit(x_train, y_train)
print(
    "RandomizedSearchCV took %.2f seconds for %d candidates parameter settings."
    % ((time() - start), n_iter_search))

C:\Users\jobao\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:285: UserWarning: The total space of parameters 6 is smaller than n_iter=15. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV took 7502.56 seconds for 15 candidates parameter settings.


In [15]:
pipeline.score(x_test, y_test)

0.24015868172108637

In [6]:
Y.dtypes
Y=Y.astype('int')
Y.dtypes

related                   int32
request                   int32
offer                     int32
aid_related               int32
medical_help              int32
medical_products          int32
search_and_rescue         int32
security                  int32
military                  int32
child_alone               int32
water                     int32
food                      int32
shelter                   int32
clothing                  int32
money                     int32
missing_people            int32
refugees                  int32
death                     int32
other_aid                 int32
infrastructure_related    int32
transport                 int32
buildings                 int32
electricity               int32
tools                     int32
hospitals                 int32
shops                     int32
aid_centers               int32
other_infrastructure      int32
weather_related           int32
floods                    int32
storm                     int32
fire    

In [88]:
pd.DataFrame(y_pred,columns=category_names)['related'].unique()

array(['1', '0'], dtype=object)

In [10]:
category_names= list(y_test.columns)
y_pred=pipeline.predict(x_test)
from sklearn.metrics import classification_report,accuracy_score

print(classification_report(pd.DataFrame(y_test), pd.DataFrame(y_pred,columns=category_names),target_names=category_names))

                        precision    recall  f1-score   support

               related       0.81      0.97      0.88      5043
               request       0.89      0.44      0.59      1164
                 offer       0.00      0.00      0.00        27
           aid_related       0.79      0.61      0.69      2742
          medical_help       0.79      0.05      0.09       546
      medical_products       0.67      0.06      0.11       324
     search_and_rescue       0.50      0.01      0.01       171
              security       0.00      0.00      0.00       108
              military       0.77      0.05      0.10       191
           child_alone       0.00      0.00      0.00         0
                 water       0.92      0.21      0.34       460
                  food       0.89      0.38      0.53       769
               shelter       0.89      0.21      0.34       567
              clothing       0.82      0.09      0.17        95
                 money       0.75      

C:\Users\jobao\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jobao\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jobao\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jobao\Anaconda3\lib\site-packages\sklearn\metrics\_cla

In [14]:
import pickle
filename = 'model.pkl'
pickle.dump(pipeline, open(filename, 'wb'))